# División de muestras

In [1]:
import joblib
import os

# Variables globales

In [2]:
FIC_ENTRADA='data_and_scaler.joblib'
RUTA_FICHEROS = r"C:\Users\jaume\Documents\Proyecto\datos"
FIC_SALIDA='train_test_val_scaler.joblib'

# Porcentajes para cada muestra
TAMANO_VALIDACION=0.1
TAMANO_TEST=0.1


In [3]:
# Carga de datos
os.chdir(RUTA_FICHEROS)

with open(FIC_ENTRADA, 'rb') as file:
    data_dict = joblib.load(file)

# Recuperar DataFrame y scaler
df_normalizado = data_dict['dataframe'].copy()  # Este DF contiene las columnas *_scaled, dummies, etc.
scaler = data_dict['scaler']              # Contiene las normalizaciones con StandarScaler()

# Dididir Train/Test/Val

Dividiré en 80% para Train, 10% para Validación y 10% para Test.

"Test" sólo lo usaré al final de todo así estarán totalmente aisladas.

Evidentemente en mi caso no puedo usar "shuffle" dado que son RNN.

In [4]:
def split_time_series(df, test_size=0.1, val_size=0.1):
    """
    Divide un DataFrame en conjuntos de entrenamiento, validación y prueba,
    manteniendo el orden temporal de los datos (sin shuffle).
    
    Parámetros:
        df (pd.DataFrame): DataFrame a dividir.
        test_size (float): Proporción de datos para el conjunto de prueba.
        val_size (float): Proporción de datos para el conjunto de validación.
    
    Retorna:
        df_train (pd.DataFrame): Conjunto de entrenamiento.
        df_val (pd.DataFrame): Conjunto de validación.
        df_test (pd.DataFrame): Conjunto de prueba.
    """
    # Definir tamaños en términos absolutos
    total_size = len(df)
    test_abs = int(total_size * test_size)
    val_abs = int(total_size * val_size)

    # División respetando el orden temporal
    df_train = df.iloc[:-test_abs-val_abs]  # Entrenamiento hasta antes de validación y test
    df_val = df.iloc[-test_abs-val_abs:-test_abs]  # Validación antes de test
    df_test = df.iloc[-test_abs:]  # Test al final

    return df_train, df_val, df_test



In [5]:
# Aplicamos la función con 10% test y 10% validación
df_train, df_val, df_test = split_time_series(df_normalizado, test_size=TAMANO_TEST, val_size=TAMANO_VALIDACION)

# Mostramos los tamaños
print(f"Tamaño Train: {len(df_train)}, Validación: {len(df_val)}, Test: {len(df_test)}")

Tamaño Train: 216768, Validación: 27095, Test: 27095


In [6]:
# Guardar en un archivo
os.chdir(RUTA_FICHEROS)

with open(FIC_SALIDA, 'wb') as file:
    joblib.dump({'train': df_train, 'test': df_test, 'val': df_val, 'scaler': scaler}, file)

